## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы __ПМ20-4 Савостьянов Александр Васильевич__

Вариант: __91__

    

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



#### Импортируем нужные библиотеки 

In [1]:
import re
import time
import json
import h5py
import nltk
import numba
import pickle
import sqlite3
import numpy as np
import pandas as pd
import dask.bag as db
import multiprocessing
import dask.array as da
import dask.dataframe as dd
from numba import vectorize
from collections import Counter
from scipy.spatial import distance
from pandarallel import pandarallel
from iteration_utilities import flatten

__Задание 1__. В файле `scores.npy` содержится массив оценок рейтингов вопросов. Напишите функцию, которая считает сумму квадратов цифр числа. Векторизуйте эту функцию при помощи `numba.vectorize` и посчитайте количество чисел из файла, сумма квадратов цифр которых меньше 50.

In [2]:
scores_data = np.load('scores.npy')
scores_data

array([21, 27, 40, ...,  0,  2,  0], dtype=int64)

In [3]:
@vectorize
def sum_squares(scores):
    splitted_numbers = (scores // (10 ** np.arange(10).reshape(10, -1))) % 10
    sums_sq = np.sum(splitted_numbers**2, axis=0)
    bool_array = bool(sums_sq < 50)
    return bool_array

In [4]:
np.sum(sum_squares(scores_data))

558363

__Задание 2__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow, в файле `answers.csv` - ответы на эти вопросы. Создайте базу данных SQLite с двумя таблицами: Questions и Answers. При создании таблиц добавьте внешний ключ между ID вопроса и ParentID ответа. Выведите на экран таблицу, содержащую ID вопроса, название вопроса и количество ответов на этот вопрос.

In [5]:
questions = pd.read_csv('questions.csv')
answers = pd.read_csv('answers.csv')

In [32]:
questions

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...
...,...,...,...,...,...,...
607277,40143190,333403.0,2016-10-19T23:36:01Z,1,How to execute multiline python code from a ba...,<p>I need to extend a shell script (bash). As ...
607278,40143228,6662462.0,2016-10-19T23:40:00Z,0,How to get google reCaptcha image source using...,<p>I understood that reCaptcha loads a new fra...
607279,40143267,4064680.0,2016-10-19T23:44:07Z,0,Updating an ManyToMany field with Django rest,<p>I'm trying to set up this API so I can use ...
607280,40143338,7044980.0,2016-10-19T23:52:27Z,2,Most possible pairs,"<p>Given a list of values, and information on ..."


In [6]:
conn = sqlite3.connect("dtz_num_2.sqlite3")
cur = conn.cursor()

In [7]:
cur.executescript("""CREATE TABLE IF NOT EXISTS Questions(
        Id INT PRIMARY KEY,
        OwnerUserId VARCHAR,
        CreationDate TEXT,
        Score INT,
        Title TEXT,
        Body TEXT
        );
                    CREATE TABLE IF NOT EXISTS Answers(
        Id INT PRIMARY KEY,
        OwnerUserId VARCHAR, 
        CreationDate TEXT,
        ParentId INT,
        Score INT,
        Body TEXT,
        FOREIGN KEY (ParentId) REFERENCES Questions(Id)
        );
        """)
conn.commit()

In [8]:
lst_tuples_questions = list(zip(*map(questions.get, questions)))
lst_tuples_answers = list(zip(*map(answers.get, answers)))

In [31]:
lst_tuples_questions

[(469,
  147.0,
  '2008-08-02T15:11:16Z',
  21,
  'How can I find the full path to a font from its display name on a Mac?',
  "<p>I am using the Photoshop's javascript API to find the fonts in a given PSD.</p>\\n\\n<p>Given a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.</p>\\n\\n<p>This is all happening in a python program running on OSX so I guess I'm looking for one of:</p>\\n\\n<ul>\\n<li>Some Photoshop javascript</li>\\n<li>A Python function</li>\\n<li>An OSX API that I can call from python</li>\\n</ul>\\n"),
 (502,
  147.0,
  '2008-08-02T17:01:58Z',
  27,
  'Get a preview JPEG of a PDF on Windows?',
  '<p>I have a cross-platform (Python) application which needs to generate a JPEG preview of the first page of a PDF.</p>\\n\\n<p>On the Mac I am spawning <a href="http://developer.apple.com/documentation/Darwin/Reference/ManPages/man1/sips.1.html">sips</a>.  Is there something similarly simple I can do on W

In [10]:
cur.executemany("INSERT OR REPLACE INTO Questions(Id, OwnerUserId, CreationDate, Score, Title, Body) VALUES(?, ?, ?, ?, ?, ?);", lst_tuples_questions)
conn.commit()

In [11]:
cur.executemany("INSERT OR REPLACE INTO Answers(Id, OwnerUserId, CreationDate, ParentId, Score, Body) VALUES(?, ?, ?, ?, ?, ?);", lst_tuples_answers)
conn.commit()

In [12]:
res = cur.execute('''SELECT Questions.Id, Title, COUNT(Answers.Id)
                    FROM Questions
                    INNER JOIN Answers
                    ON Questions.Id = Answers.ParentId
                    GROUP BY Questions.Id''')

table = res.fetchall()
pd.DataFrame(table, columns=["Question_id", "Title", "Amount_of_answ"])

,Question_id,Title,Amount_of_answ
0,469,How can I find the full path to a font from it...,4
1,502,Get a preview JPEG of a PDF on Windows?,3
2,535,Continuous Integration System for a Python Cod...,7
3,594,cx_Oracle: How do I iterate over a result set?,3
4,683,Using 'in' to match an attribute of Python obj...,8
...,...,...,...
539233,40142906,can't assign to function call Error-Python,2
539234,40142948,How do you make it so that a function returns ...,1
539235,40143133,error handling with BeautifulSoup when scraped...,1
539236,40143166,finding cubed root using delta and epsilon in ...,1


In [13]:
cur.close()
conn.close()

__Задание 3__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Используя `multiprocessing.Pool` выясните, сколько вопросов задали в 2011 году. При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.

In [33]:
def parallel_map(s: pd.Series, f: callable) -> pd.Series:
    s_split = np.array_split(s, 8)
    split_bool = []
    for df_sp in range(len(s_split)):
        with multiprocessing.Pool(processes=len(s_split)) as pool:
            split_bool.append(list(pool.map(f, s_split[df_sp].CreationDate)))
    series_flat_split_bool = pd.Series(list(flatten(split_bool)))
    return series_flat_split_bool

In [34]:
%%file dtz_num_3.py
import re

def f(tag: str) -> bool:
    patt = re.compile(r"(?:2011)")
    find = patt.findall(tag)
    res = False if not find else True
    return res

Overwriting dtz_num_3.py


In [35]:
from dtz_num_3 import f

np.sum(parallel_map(pd.read_csv('questions.csv'), f))

40910

__Задание 4__. В файле `qembeddings.hdf` находятся векторные представления заголовков вопросов, заданные на сайте stackoverflow. В файле `qtitles.pickle` находится список заголовков этих вопросов. Напишите функцию, которая для вопроса с индексом `i` выводит заголовки 5 ближайших вопросов в смысле расстояния между эмбеддингами этих вопросов. Выберите метрику, наиболее подходящую (на ваш взгляд) для решения этой задачи. Для работы с эмбеддингами используйте `dask.array`.

In [17]:
hdf = h5py.File("qembeddinbgs.h5", "r")
dset = hdf["embeddings"]
arr_embeddings = da.from_array(dset)
arr_embeddings

dask.array<array, shape=(600000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [18]:
with open(
    "qtitles.pickle",
    "rb"
) as fp:
    qtitles_lst = pickle.load(fp)
qtitles_lst

['POST request failing silently',
 'Setting timezone for timestamp data in pandas dataframe',
 'Hue supervisor startup error hadoop working fine - Python packages in hue not working',
 'sum variable in a range (python)',
 'Table with missing <tr> around a row: can I parse it?',
 'WTForms FieldList with optional fields',
 'Understanding PHP Arrays from a Python background',
 'Raspberry Pi I2C OSError',
 'Check for pending Django migrations',
 'Error Code 1 While Loading Python Pillow Module',
 'Relationship extraction from text using python',
 'Modifying and creating sites in djangos admin app',
 'How to print data into a specific json format in python',
 'ImportError installing GDAL on Ubuntu 14.04 with Anaconda Python 3.5',
 'get mean for all columns in a dataframe and create a new dataframe',
 'Python openpyxl fails to save existing excel having customized column width',
 'How to capture website screenshot in high resolution?',
 'Build a bucket-list histogram from scratch in Python',

In [19]:
def cosine_distance(ind, am_of_q, titles): # индекс, количество вопросов, список заголовков
    cos_sim = 1 - (((arr_embeddings[ind]*arr_embeddings).sum(axis=1))/(((arr_embeddings[ind]**2).sum())*((arr_embeddings**2).sum(axis=1))))
    cos_sim_closest = cos_sim.argtopk(-(am_of_q+1))[1:].compute()
    list_of_closest = list(map(lambda i: titles[i], cos_sim_closest))
    return list_of_closest

In [30]:
index = 15
print(qtitles_lst[index])
cosine_distance(index, 5, qtitles_lst) 

Python openpyxl fails to save existing excel having customized column width


['Python SQLite avoid overcrowding by deleting the last item',
 'Python fails to parse objects list from valid yaml',
 'Python storeAsUrl fails to save document',
 'Python-boto fails to create image from an instance',
 'Error in opening Python executable file, created using cx_freeze']

__Задание 5__. В файлах `questions_with_answers_*.jsonl` содержатся вопросы, заданные на сайте stackoverflow, и ответы на них. Разбейте вопросы на 3 группы: вопросы, на которые дали меньше 5 ответов; вопросы, на которые дали больше 5 ответов, но меньше 15 ответов; и вопросы, на которые дали 15 ответов или более. Посчитайте количество вопросов в каждой из групп. Для обработки данных используйте `dask bag`.

##### если делать через read_text то вызывается ошибка после применения compute

In [22]:
def read_json_file_1(filename):
    with open(filename, encoding='utf-8') as f:
        lines = f.read().splitlines()

    lst = []
    for i, line in enumerate(lines):
        try:
            lst.append(json.loads(line))
        except:
            pass
    
    return lst

In [23]:
bag_mapped_func = db.from_sequence(['questions_with_answers_aa.jsonl', 
                            'questions_with_answers_ab.jsonl', 
                            'questions_with_answers_ac.jsonl', 
                            'questions_with_answers_ad.jsonl', 
                            'questions_with_answers_af.jsonl', 
                            'questions_with_answers_ae.jsonl', 
                            'questions_with_answers_ag.jsonl', 
                            'questions_with_answers_ah.jsonl']).map(read_json_file_1)

bag_tuple = tuple(flatten(bag_mapped_func.compute()))
bag_tuple

({'Id': 25486492,
  'OwnerUserId': 2384800.0,
  'CreationDate': '2014-08-25T12:57:02Z',
  'Score': 0,
  'Title': 'Using certain rows as columns to create DataFrame',
  'Body': "<p>I have a data frame <em>DF</em> looking like</p>\n\n<pre><code>DF =\n\n     Date            Counterparty    Amount\n0    2014-08-25      A               100\n1    2014-08-25      A               200   \n2    2014-08-25      B               300   \n3    2014-08-26      C               400 \n4    2014-08-27      C               500  \n5    2014-08-27      A               600\n6    2014-08-29      D               1000\n</code></pre>\n\n<p>I want to somehow create a new data frame <em>DF_new</em>, with (unique) columns as of the rows of <em>DF</em>:s column <em>'Counterparty'</em> with aggregated amounts from column <em>'Amount'</em>. In code, a result looking like</p>\n\n<pre><code>DF_new =\n\n     Date            A            B            C            D\n0    2014-08-25      300          300          0         

In [24]:
bag = db.from_sequence(bag_tuple)
bag.take(5)

({'Id': 25486492,
  'OwnerUserId': 2384800.0,
  'CreationDate': '2014-08-25T12:57:02Z',
  'Score': 0,
  'Title': 'Using certain rows as columns to create DataFrame',
  'Body': "<p>I have a data frame <em>DF</em> looking like</p>\n\n<pre><code>DF =\n\n     Date            Counterparty    Amount\n0    2014-08-25      A               100\n1    2014-08-25      A               200   \n2    2014-08-25      B               300   \n3    2014-08-26      C               400 \n4    2014-08-27      C               500  \n5    2014-08-27      A               600\n6    2014-08-29      D               1000\n</code></pre>\n\n<p>I want to somehow create a new data frame <em>DF_new</em>, with (unique) columns as of the rows of <em>DF</em>:s column <em>'Counterparty'</em> with aggregated amounts from column <em>'Amount'</em>. In code, a result looking like</p>\n\n<pre><code>DF_new =\n\n     Date            A            B            C            D\n0    2014-08-25      300          300          0         

In [26]:
counts_less_five = bag.map(lambda x: len(x.get('answers')) <= 5).compute()
counts_less_five.count(True)

392186

In [27]:
counts_more_or_equal_fithteen = bag.map(lambda x: len(x.get('answers')) >= 15).compute()
counts_more_or_equal_fithteen.count(True)

199

In [28]:
counts_five_to_fithteen = bag.map(lambda x: 15 > len(x.get('answers')) > 5).compute()
counts_five_to_fithteen.count(True)

6780

__Задание 6__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Для вопросов, рейтинг которых больше медианного значения по столбцу, найдите топ-5 самых часто используемых слов во всех вопросах в сокупности. Для обработки данных используйте `dask.dataframe`.

In [39]:
questions_dd = dd.read_csv("questions.csv", parse_dates=["CreationDate"])
questions_dd = questions_dd[questions_dd["Score"] > questions_dd.Score.mean()]

tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
all_words = questions_dd.Body.apply(lambda x: tokenizer.tokenize(x), meta=('Body', 'object'))
all_words.compute()

0        [p, I, am, using, the, Photoshop, s, javascrip...
1        [p, I, have, a, cross, platform, Python, appli...
2        [p, I, m, starting, work, on, a, hobby, projec...
3        [p, There, are, several, ways, to, iterate, ov...
4        [p, I, don, t, remember, whether, I, was, drea...
                               ...                        
38703    [p, Now, that, code, __init__, py, code, is, a...
38726    [p, I, m, trying, to, read, in, em, near, real...
38818    [p, I, frequently, see, code, like, this, at, ...
38819    [p, Ultimately, I, want, to, remove, all, expl...
38861    [p, consider, the, code, df, code, p, n, n, pr...
Name: Body, Length: 110573, dtype: object

In [40]:
words = list(flatten(all_words.compute()))
Counter(words).most_common(5)

[('n', 2644012),
 ('p', 1020190),
 ('code', 732609),
 ('the', 571521),
 ('I', 514508)]